In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## Langsmith Tracking and Tracing
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="o1-mini")
print("Model Name:", llm)

Model Name: client=<openai.resources.chat.completions.completions.Completions object at 0x000001D2E16AABA0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D2E16AB620> root_client=<openai.OpenAI object at 0x000001D2E1551E80> root_async_client=<openai.AsyncOpenAI object at 0x000001D2E16AACF0> model_name='o1-mini' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="qwen-qwq-32b")
model.invoke("What is the capital of France?")

AIMessage(content="\n<think>\nOkay, I need to figure out what the capital of France is. Let's start by recalling what I know about France. France is a country in Western Europe, right? I remember learning about some of its cities like Paris, Lyon, Marseille, and maybe Nice. I think Paris is a major city there. Wait, isn't Paris the capital? Hmm, but I should make sure I'm not confusing it with another country's capital. Let me think... Spain's capital is Madrid, Italy's is Rome, so France must have a different one. I've heard the Eiffel Tower is in Paris, which is in France. That's a landmark, so maybe that's in the capital. But wait, sometimes big cities aren't the capital. Like in the US, New York is big but the capital is Washington D.C. But in France's case, I think Paris is both the largest city and the capital. Let me try to remember any historical context. The French government is there, right? The President's palace, maybe the Élysée Palace? That's in Paris. Also, when I see ne

In [7]:
### Prompt Engineering
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert AI Engineer. Provide a detailed answer."),
    ("user", "{input}"),
])
prompt 

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide a detailed answer.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [8]:
### Chaining: getting chained the prompt and model together

chain = prompt | model
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide a detailed answer.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D2E1B19010>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D2E1B1A120>, model_name='qwen-qwq-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
response = chain.invoke({"input": "What is the capital of France?"})
print("Response:", response.content)

Response: 
<think>
Okay, the user is asking for the capital of France. Let me start by recalling basic geography. France is a country in Western Europe. The capital cities are usually the political and administrative centers of a country. Paris is the most well-known city in France. Wait, but maybe there's some trick here? Let me think. Sometimes people confuse Paris with other cities, but no, I'm pretty sure Paris has been the capital for a long time. The government, including the President's office, is there. Maybe check if there was a historical change? Like, during some events, but no, even during wartime, Paris remained the capital. Also, in terms of cultural significance, Paris is the heart of France. So the answer should be Paris. I don't see any reason to doubt that. Just to be thorough, maybe think about other possibilities. Lyon? No, that's a major city but not the capital. Marseille? No. So yes, the answer is definitely Paris. I should present that clearly.
</think>

The cap

In [12]:
### Output Parsing : How the ouput will be visible to the user
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser
response = chain.invoke({"input": "What is the capital of France?"})
print("Parsed Response:", response)

Parsed Response: 
<think>
Okay, the user is asking, "What is the capital of France?" Let me start by recalling basic geography. France is a country in Western Europe, and I know that Paris is its capital. But wait, maybe I should double-check to be sure. Sometimes people confuse capitals, like thinking the capital of the UK is London, which it is, but maybe there's a trick here. No, France's capital is definitely Paris. Let me think if there's any recent change? I don't recall any such thing. Historically, Paris has been the capital for a long time. Maybe the user is a student needing this for a test, or just a general question. I should confirm through my knowledge base. Yes, all sources indicate Paris as the capital. To be thorough, I can mention a bit more context, like Paris being a major cultural and political center. But the answer should be straightforward. Just need to present it clearly and concisely. Alright, no complications here. The answer is Paris.
</think>

The capital o